<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>

<div style="float:right; width:98 px; height:98px;">
<img src="https://media.licdn.com/dms/image/C4E0BAQFlOZSAJABP4w/company-logo_200_200/0/1548285168598?e=2147483647&v=beta&t=g4jl8rEhB7HLJuNZhU6OkJWHW4cul_y9Kj_aoD7p0_Y" alt="STI Logo" style="height: 98px;">
</div>


<h1>Calculate Windspeed and Direction for Each Model and Timestep</h1>
By: Kayla Besong, PhD
    <br>
Last Edited: 12/11/23
<br>
<br>    
<br>
Takes UFS S2S data and calculates the windspeed and direction. File_concat_mod_functions.ipynb imports the function that computes the 24HR AVG, MIN, MAX outputs. 
<div style="clear:both"></div>
</div>

<hr style="height:2px;">

## Import needed libraries, etc.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from dask.distributed import Client, LocalCluster
import dask.array as da
import os
import glob
from metpy.units import units
import math
import metpy

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

## Establish a dask client. This is a lot of data.

In [ ]:
Cluster = LocalCluster(n_workers = 8, threads_per_worker=4, memory_limit='30GB',  processes=True)
#Cluster = LocalCluster()

In [ ]:
client = Client(Cluster)
client

### The integral notebook of functions to run

In [ ]:
%run File_concat_mod_functions.ipynb

the main function to run is:

    return_concat(model, variable, prototype = None)

Where the prototype is only needed for the UFS_S2S model choice. 

## UFS
expected timesteps = 1460

In [ ]:
model = 'UFS_S2S'
protoypes = [6,7,8]

In [ ]:
var_list = get_model_var(model,'WIND')[0:2]
var_list

In [ ]:
#parent_dirs = glob.glob(os.path.join(f'{model}/{p}/', '*'))


In [ ]:
%%time
for p in protoypes:

    parent_dirs = glob.glob(os.path.join(f'{model}/{p}/', '*'))

    for i in parent_dirs:
                
        dir_maker(os.path.join(f'{output_dir}', f'{i}'))
        
        u_files = glob.glob(os.path.join(f'{i}/{var_list[0]}', f'*{var_list[0]}*'))
        v_files = glob.glob(os.path.join(f'{i}/{var_list[1]}', f'*{var_list[1]}*'))
    
        u_file_cc = xr.open_mfdataset(u_files, combine = 'nested', concat_dim = 'valid_time', chunks = get_chunk(model))
        v_file_cc = xr.open_mfdataset(v_files, combine = 'nested', concat_dim = 'valid_time', chunks = get_chunk(model))
        
        u_file_cc = u_file_cc.sortby('valid_time')
        v_file_cc = v_file_cc.sortby('valid_time')
    
        w_speed_ufs = metpy.calc.wind_speed(u_file_cc.u10 * units.meter_per_second, v_file_cc.v10 * units.meter_per_second)
        w_speed_ufs = w_speed_ufs.metpy.dequantify().to_dataset(name = 'wspeed')
    
        wind_direction_ufs = (270 - np.degrees(np.arctan2(v_file_cc.v10, u_file_cc.u10))) % 360
        wind_direction_ufs = wind_direction_ufs.to_dataset(name = 'wdir')
        
        wind_direction_ufs = wind_direction_ufs.drop({'heightAboveGround'})
        w_speed_ufs = w_speed_ufs.drop({'heightAboveGround'})
        
        resampler_UFS('wspeed', w_speed_ufs.chunk(get_chunk(model)), output_dir, i, p)
        resampler_UFS('wdir', wind_direction_ufs.chunk(get_chunk(model)), output_dir, i, p)
